In [2]:
!pip install pyswarms

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import numpy as np
import pyswarms as ps
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def logreg_fitness(params):
    C = 10 ** (params[:, 0] * 3 - 4) 
    penalty_idx = np.round(params[:, 1] * 3).astype(int) 
    penalties = ['l1', 'l2', 'elasticnet', None]
    
    scores = []
    for i in range(params.shape[0]):
        try:
            model = LogisticRegression(
                C=C[i],
                penalty=penalties[penalty_idx[i]],
                solver='saga' if penalties[penalty_idx[i]] in ['l1', 'elasticnet'] else 'lbfgs',
                max_iter=1000,
                random_state=42
            )
            score = cross_val_score(model, X_train_scaled, y_train, cv=3, scoring='accuracy').mean()
            scores.append(score)
        except:
            scores.append(0)  
            
    return -np.array(scores)  

In [ ]:
bounds = (np.array([0, 0]), np.array([1, 1]))

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 10, 'p': 2}

optimizer = ps.single.GlobalBestPSO(
    n_particles=20,
    dimensions=2,
    options=options,
    bounds=bounds
)

best_cost, best_params = optimizer.optimize(logreg_fitness, iters=30)

best_C = 10 ** (best_params[0] * 3 - 4)
penalties = ['l1', 'l2', 'elasticnet', None]
best_penalty = penalties[int(round(best_params[1] * 3))]

print(f"Лучшие параметры: C={best_C:.4f}, penalty={best_penalty}")

best_logreg = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    solver='saga' if best_penalty in ['l1', 'elasticnet'] else 'lbfgs',
    max_iter=1000,
    random_state=42
).fit(X_train_scaled, y_train)

2025-05-03 09:55:23,907 - pyswarms.single.global_best - INFO - Optimize for 30 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 10, 'p': 2}
pyswarms.single.global_best:   0%|          |0/30/Users/macbook/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/macbook/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/macbook/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/macbook/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/macbook/Library/Python/3.9/lib/pytho

Лучшие параметры: C=0.0568, penalty=None


/Users/macbook/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [ ]:
def tree_fitness(params):
    max_depth = np.round(params[:, 0] * 49 + 1).astype(int)  
    min_samples_split = np.round(params[:, 1] * 18 + 2).astype(int) 
    min_samples_leaf = np.round(params[:, 2] * 9 + 1).astype(int) 
    criterion = np.round(params[:, 3]).astype(int) 
    criteria = ['gini', 'entropy']
    
    scores = []
    for i in range(params.shape[0]):
        model = DecisionTreeClassifier(
            max_depth=max_depth[i],
            min_samples_split=min_samples_split[i],
            min_samples_leaf=min_samples_leaf[i],
            criterion=criteria[criterion[i]],
            random_state=42
        )
        score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
        scores.append(score)
        
    return -np.array(scores) 

In [ ]:
bounds = (np.array([0, 0, 0, 0]), np.array([1, 1, 1, 1]))

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 10, 'p': 2}

optimizer = ps.single.GlobalBestPSO(
    n_particles=20,
    dimensions=4,
    options=options,
    bounds=bounds
)

best_cost, best_params = optimizer.optimize(tree_fitness, iters=30)

best_max_depth = int(round(best_params[0] * 49 + 1))
best_min_samples_split = int(round(best_params[1] * 18 + 2))
best_min_samples_leaf = int(round(best_params[2] * 9 + 1))
best_criterion = ['gini', 'entropy'][int(round(best_params[3]))]

print(f"Лучшие параметры: max_depth={best_max_depth}, min_samples_split={best_min_samples_split}, "
      f"min_samples_leaf={best_min_samples_leaf}, criterion={best_criterion}")

best_tree = DecisionTreeClassifier(
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    min_samples_leaf=best_min_samples_leaf,
    criterion=best_criterion,
    random_state=42
).fit(X_train, y_train)

2025-05-03 09:56:17,203 - pyswarms.single.global_best - INFO - Optimize for 30 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 10, 'p': 2}
pyswarms.single.global_best: 100%|██████████|30/30, best_cost=-0.95
2025-05-03 09:56:18,872 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.9500000000000001, best pos: [0.09944094 0.31284728 0.07320279 0.80389527]


Лучшие параметры: max_depth=6, min_samples_split=8, min_samples_leaf=2, criterion=entropy


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_logreg = best_logreg.predict(X_test_scaled)
print("Logistic Regression:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_logreg):.4f}")
print(classification_report(y_test, y_pred_logreg))

y_pred_tree = best_tree.predict(X_test)
print("\nDecision Tree:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tree):.4f}")
print(classification_report(y_test, y_pred_tree))

Logistic Regression:
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


Decision Tree:
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [ ]:
import joblib

joblib.dump(best_logreg, 'best_logreg_pso.joblib')
joblib.dump(scaler, 'scaler.joblib')  

joblib.dump(best_tree, 'best_tree_pso.joblib')